In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
from transformers import (
                        pipeline, 
                        AutoTokenizer, 
                        AutoModelForCausalLM
                        )
import torch
import pandas as pd
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from dotenv import load_dotenv
import warnings
import s3fs

/opt/conda/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
load_dotenv()
pd.set_option("display.max_columns", None)
warnings.simplefilter("ignore")
fs = s3fs.S3FileSystem(
            client_kwargs={"endpoint_url": "https://minio.lab.sspcloud.fr"},
            key=os.environ["Accesskey"],
            secret=os.environ["Secretkey"],
            token=os.environ["Token"]
)

In [4]:
with fs.open("elissamim/text_classification_men/data/stages-votes.json", "r") as file:
    df = pd.read_json(file)

df = df.groupby("phrase_text", as_index = False)["sol"].apply(lambda x: x.mode().iloc[0])
df["sol"]=df["sol"].apply(lambda x: 1 if x == "ok" else 0)

In [5]:
model_name = "mistralai/Mistral-7B-Instruct-v0.1"

tokenizer = AutoTokenizer.from_pretrained(model_name)

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="auto",
    torch_dtype=torch.float16,
)

# Create a text generator
generator = pipeline(
    "text-generation",
    model = model,
    tokenizer = tokenizer,
)

def few_shot_classification(text:str):

    prompt = f"""
    Tu es un expert en orientation scolaire. Ta tâche est de déterminer si un intitulé de stage de 3ᵉ est acceptable pour un collégien. Réponds uniquement par "oui" ou "non".
    
    Exemples :

    Example 1:
    Intitulé : "Stage dans une mairie"  
    Acceptable ? oui

    Example 2:
    Intitulé : "Stage chez un tatoueur"  
    Acceptable ? non

    Example 3:
    Intitulé : "Stage d’observation dans un cabinet vétérinaire"  
    Acceptable ? oui

    Example 4:
    Intitulé : "Stage dans une boîte de nuit"  
    Acceptable ? non

    Example 5:
    Intitulé : "Aide à la comptabilité dans une brasserie artisanale"
    Acceptable ? oui

    Example 6:
    Intitulé : "Aide à un sommelier pour la dégistation et l'évaluation de vins"
    Acceptable ? non
    
    Maintenant, analyse le nouvel intitulé :
    
    Intitulé : "{text}"  
    Acceptable ?
    """

    return generator(
        prompt,
        max_new_tokens=10,
        do_sample=False
    )


Loading checkpoint shards: 100%|██████████| 2/2 [00:54<00:00, 27.33s/it]
Device set to use cpu


In [6]:
few_shot_classification("Stage dans un laboratoire d'analyse par chromatographie de prélèvements biologiques")

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


[{'generated_text': '\n    Tu es un expert en orientation scolaire. Ta tâche est de déterminer si un intitulé de stage de 3ᵉ est acceptable pour un collégien. Réponds uniquement par "oui" ou "non".\n\n    Exemples :\n\n    Example 1:\n    Intitulé : "Stage dans une mairie"  \n    Acceptable ? oui\n\n    Example 2:\n    Intitulé : "Stage chez un tatoueur"  \n    Acceptable ? non\n\n    Example 3:\n    Intitulé : "Stage d’observation dans un cabinet vétérinaire"  \n    Acceptable ? oui\n\n    Example 4:\n    Intitulé : "Stage dans une boîte de nuit"  \n    Acceptable ? non\n\n    Example 5:\n    Intitulé : "Aide à la comptabilité dans une brasserie artisanale"\n    Acceptable ? oui\n\n    Example 6:\n    Intitulé : "Aide à un sommelier pour la dégistation et l\'évaluation de vins"\n    Acceptable ? non\n\n    Maintenant, analyse le nouvel intitulé :\n\n    Intitulé : "Stage dans un laboratoire d\'analyse par chromatographie de prélèvements biologiques"  \n    Acceptable ?\n    \n    Répo

In [7]:
few_shot_classification("Stage de vente de bières artisanales")

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


[{'generated_text': '\n    Tu es un expert en orientation scolaire. Ta tâche est de déterminer si un intitulé de stage de 3ᵉ est acceptable pour un collégien. Réponds uniquement par "oui" ou "non".\n\n    Exemples :\n\n    Example 1:\n    Intitulé : "Stage dans une mairie"  \n    Acceptable ? oui\n\n    Example 2:\n    Intitulé : "Stage chez un tatoueur"  \n    Acceptable ? non\n\n    Example 3:\n    Intitulé : "Stage d’observation dans un cabinet vétérinaire"  \n    Acceptable ? oui\n\n    Example 4:\n    Intitulé : "Stage dans une boîte de nuit"  \n    Acceptable ? non\n\n    Example 5:\n    Intitulé : "Aide à la comptabilité dans une brasserie artisanale"\n    Acceptable ? oui\n\n    Example 6:\n    Intitulé : "Aide à un sommelier pour la dégistation et l\'évaluation de vins"\n    Acceptable ? non\n\n    Maintenant, analyse le nouvel intitulé :\n\n    Intitulé : "Stage de vente de bières artisanales"  \n    Acceptable ?\n    \n    Réponse : oui'}]

In [8]:
few_shot_classification("Stage de consommation de bières artisanales, pour en évaluer la qualité")

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


[{'generated_text': '\n    Tu es un expert en orientation scolaire. Ta tâche est de déterminer si un intitulé de stage de 3ᵉ est acceptable pour un collégien. Réponds uniquement par "oui" ou "non".\n\n    Exemples :\n\n    Example 1:\n    Intitulé : "Stage dans une mairie"  \n    Acceptable ? oui\n\n    Example 2:\n    Intitulé : "Stage chez un tatoueur"  \n    Acceptable ? non\n\n    Example 3:\n    Intitulé : "Stage d’observation dans un cabinet vétérinaire"  \n    Acceptable ? oui\n\n    Example 4:\n    Intitulé : "Stage dans une boîte de nuit"  \n    Acceptable ? non\n\n    Example 5:\n    Intitulé : "Aide à la comptabilité dans une brasserie artisanale"\n    Acceptable ? oui\n\n    Example 6:\n    Intitulé : "Aide à un sommelier pour la dégistation et l\'évaluation de vins"\n    Acceptable ? non\n\n    Maintenant, analyse le nouvel intitulé :\n\n    Intitulé : "Stage de consommation de bières artisanales, pour en évaluer la qualité"  \n    Acceptable ?\n    \n    Réponse : oui'}]

In [9]:
few_shot_classification("Stage de consommation d'alcool, pour en évaluer la qualité")

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


[{'generated_text': '\n    Tu es un expert en orientation scolaire. Ta tâche est de déterminer si un intitulé de stage de 3ᵉ est acceptable pour un collégien. Réponds uniquement par "oui" ou "non".\n\n    Exemples :\n\n    Example 1:\n    Intitulé : "Stage dans une mairie"  \n    Acceptable ? oui\n\n    Example 2:\n    Intitulé : "Stage chez un tatoueur"  \n    Acceptable ? non\n\n    Example 3:\n    Intitulé : "Stage d’observation dans un cabinet vétérinaire"  \n    Acceptable ? oui\n\n    Example 4:\n    Intitulé : "Stage dans une boîte de nuit"  \n    Acceptable ? non\n\n    Example 5:\n    Intitulé : "Aide à la comptabilité dans une brasserie artisanale"\n    Acceptable ? oui\n\n    Example 6:\n    Intitulé : "Aide à un sommelier pour la dégistation et l\'évaluation de vins"\n    Acceptable ? non\n\n    Maintenant, analyse le nouvel intitulé :\n\n    Intitulé : "Stage de consommation d\'alcool, pour en évaluer la qualité"  \n    Acceptable ?\n    \n    Réponse : oui'}]

In [ ]:
df["sol_generated"] = df["phrase_text"].apply(lambda x: few_shot_classification(x))
df["sol_generated"] = df["sol_generated"].apply(lambda x: 1 if x == "oui" else 0)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for o

In [ ]:
accuracy_score(df["sol"], df["sol_generated"])

In [ ]:
classification_report(df["sol"], df["sol_generated"])

In [ ]:
confusion_matrix(df["sol"], df["sol_generated"])